In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import solve
from utility import orbit, BrusselatorModel
import sys
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigs
from matplotlib.backends.backend_pdf import PdfPages


In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_2.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    print("Loaded parameters:", model.N)

f = model.dydt
Jacf = model.brusselator_jacobian
# Parameters
z_L = model.z_L
N = model.N
A, B = model.A, model.B
z = np.linspace(0, z_L, N)
perturb = np.sin(np.pi*(z/z_L))

X0 = A + 0.01*perturb
Y0 = B/A + 0.01*perturb

y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

Max_iter = 30
epsilon = model.precision
T_0 = model.T_ini


t_eval = np.linspace(0.0,20*model.T_ini, 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 20*T_0],
                t_eval=[20*model.T_ini], 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]
y_T.shape


In [ ]:
orbit_finder = orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)

k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy = orbit_finder.Newton_orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)

T = T_by_iter[k-1]
y = np.array(y_by_iter[k-1])


In [ ]:
p0, pe = 5 ,4

subspace_iter =4
rho = 0.5
eig, eigvec = np.linalg.eig(monodromy)
# V_0 = np.real(eigvec[:p0+pe])
V_0 = np.eye(len(y_T))[:,:p0+pe]
V_0,_ = np.linalg.qr(V_0)
v0 = V_0[:,0]

### Checking the subspace iteration algorithm

In [ ]:
%%time
eigen , Vp = orbit_finder.base_Vp(v0, y, T, f, Jacf, p0+pe, epsilon)
Re, Ye, Ve, We,p =  orbit_finder.subsp_iter_projec(V_0, y, T, f, Jacf,rho, p0, pe, subspace_iter, tol=1e-6)

In [ ]:
# Vp,_ = np.linalg.qr(Ve @ Ye[:, :p])
Vp = Ve @ Ye[:, :p]
# Vp,_ = np.linalg.qr(Ve @ Ye[:, :p])
print("Orthogonality :", np.allclose(Vp.T@Vp, np.eye(Vp.shape[1])))
np.linalg.norm(Vp[:,0],ord=2)


In [ ]:
eigen, _ = np.linalg.eig(Re)
p

In [ ]:
mask1 = np.abs(eig) > rho
mask2 = np.abs(eigen) > rho
max_eigenvalues = np.sort(eig[mask1])[::-1]   
print("Eigenvalues from subspace iteration \n",(eigen[mask2]))
print("The first eigenvalues of the converged monodromy matrix \n", max_eigenvalues)
np.abs(eigen[mask2])

### Newton-Picard Gauss-Seidel 

In [ ]:
%%time
# k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy = orbit_finder.Newton_Picard1(f,y_T,
# T_0, v0, p0,pe, rho, Jacf, Max_iter, epsilon)
k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy = orbit_finder.Newton_Picard_subspace_iter(f,y_T,
                         T_0, V_0, p0,pe, rho, Jacf, Max_iter,subspace_iter, epsilon)

## Convergence check

In [ ]:
eig, eigvec = np.linalg.eig(monodromy)
# Extract real and imaginary parts of eigenvalues
real_parts = np.real(eig)
imaginary_parts = np.imag(eig)
print(imaginary_parts.shape)
# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the unit circle
theta = np.linspace(0, 2 * np.pi, 1000)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax.plot(circle_x, circle_y, 'k--', label='Unit Circle')

# Plot the eigenvalues
ax.scatter(real_parts, imaginary_parts, color='r', label='Eigenvalues')

# Set labels and title
ax.set_xlabel(r'Re($\lambda$)')
ax.set_ylabel(r'Im($\lambda$)')
ax.set_title(f'Eigenvalues of the Monodromy matrix on Complex Plane \n with L = {model.L}')

# Set equal aspect ratio
ax.set_aspect('equal', 'box')

# Add grid, legend, and plot
ax.grid(True)
# ax.legend()

# Show the plot
plt.show()

Tab = np.asarray(y_by_iter[:k-1])
X = Tab[:,N-2:]
Y = Tab[:,:N-2]

fig0, ax = plt.subplots(2,2,sharex='all')
ax[0,0].plot(np.arange(k-1),X.mean(axis=1),'+-')
ax[0,0].set_ylabel(f"$<X>$")

ax[0,1].plot(np.arange(k-1),Y.mean(axis=1),'+-')
ax[0,1].set_ylabel(f"$<Y>$")
ax[1,0].semilogy(np.arange(k),Norm_Deltay[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(r"$\parallel \Delta y \parallel$")
ax[1,1].semilogy(np.arange(k),Norm_B[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(r"$\parallel \phi(y^*(0),T) - y^*(T) \parallel$")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig0.set_size_inches((8,8))
fig0.suptitle(f'Brusselator model: $L=%.4f$ \n $T = %.4f$ ' % (model.L, T_by_iter[k-1]))
fig0.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator/NP_test2_m_{str(m)}.png')

plt.show()
y0 = np.array(y_by_iter[k-1])
t_eval = np.linspace(0.0,2*T_by_iter[k-1], 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 2*T_by_iter[k-1]],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

Xmean = sol.y[:N-2,:]
#Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
#Ymean = np.mean(Ymean,axis = 0)
fig1, ax = plt.subplots(1,2)
ax[0].plot(t_eval, Xmean[23], label = '<X>')
ax[0].plot(t_eval, Ymean[23], label='<Y>')
ax[0].legend()
ax[0].set_ylabel("Concentrations")
ax[0].set_xlabel("t")
ax[1].plot(Xmean[23],Ymean[23])
ax[1].set_ylabel(r"$<Y>$")
ax[1].set_xlabel(r"$<X>$")
fig1.set_size_inches((10,5))
fig1.suptitle(r'Brusselator Model with Dirichlet BCs:')
fig1.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')

#_____Saving the graphs into a pdf file__________
# ficout = model.out_dir + "test_NP_Arnoldi_%i.pdf" %model.num_test

# with PdfPages(ficout, keep_empty=False) as pdf:
#         pdf.savefig(figure=fig)
#         pdf.savefig(figure=fig0)
#         pdf.savefig(figure=fig1)